# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [43]:
import numpy as np
import pandas as pd
import math
import requests
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [162]:
symbol ='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2086448680023,
 'week52high': 134.54,
 'week52low': 57.43,
 'week52change': 0.817718610816299,
 'sharesOutstanding': 17744686649,
 'float': 0,
 'avg10Volume': 91425477,
 'avg30Volume': 96227889,
 'day200MovingAvg': 111.28,
 'day50MovingAvg': 123.76,
 'employees': 0,
 'ttmEPS': 3.34,
 'ttmDividendRate': 0.8337576357782243,
 'dividendYield': 0.00689050126177925,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-05',
 'nextEarningsDate': '0',
 'peRatio': 36.626675276114064,
 'beta': 1.1915746975250852,
 'maxChangePercent': 46.58827229464723,
 'year5ChangePercent': 3.6739908557727046,
 'year2ChangePercent': 1.990256564680276,
 'year1ChangePercent': 0.843127912625261,
 'ytdChangePercent': 0.6962857064807938,
 'month6ChangePercent': 0.455785453839425,
 'month3ChangePercent': 0.09598378277955526,
 'month1ChangePercent': 0.027171929623134597,
 'day30ChangePercent': 0.02804636003639743,
 'day5ChangePercent': 0.0013659457576635059}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [163]:
#price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

36.626675276114064

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [17]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append (
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )
#data
final_dataframe 

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.13,0.433134,N/A
1,AAL,17.27,-0.367024,N/A
2,AAP,160.15,0.018741,N/A
3,AAPL,123.73,0.861111,N/A
4,ABBV,109.09,0.296535,N/A
...,...,...,...,...
95,CINF,82.26,-0.216022,N/A
96,CL,86.71,0.295238,N/A
97,CLX,208.37,0.372455,N/A
98,CMA,54.59,-0.203672,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [22]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dateframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.94,2.124126,N/A
1,31,AMD,95.92,1.381775,N/A
2,23,ALB,141.73,1.165809,N/A
3,3,AAPL,123.73,0.861111,N/A
4,24,ALGN,525.89,0.847084,N/A
...,...,...,...,...,...
95,81,CBOE,89.30,-0.262226,N/A
96,19,AIV,41.70,-0.332058,N/A
97,55,BA,230.65,-0.348773,N/A
98,1,AAL,17.27,-0.367024,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [24]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    
    try: 
        float(portfolio_size)
    except ValueError:
        print ('That is  not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio10000000
10000000


In [29]:
position_size = float(portfolio_size)/len(final_dataframe.index)
#position_size
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe
    

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.94,2.124126,2568
1,31,AMD,95.92,1.381775,1042
2,23,ALB,141.73,1.165809,705
3,3,AAPL,123.73,0.861111,808
4,24,ALGN,525.89,0.847084,190
...,...,...,...,...,...
95,81,CBOE,89.30,-0.262226,1119
96,19,AIV,41.70,-0.332058,2398
97,55,BA,230.65,-0.348773,433
98,1,AAL,17.27,-0.367024,5790


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [149]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

# change output
pd.options.display.max_rows = 510

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series (
                [
                        symbol,
                        data[symbol]['price'],
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A',
                        'N/A'

                ],
                index = hqm_columns
            ),
            ignore_index=True
        )
hqm_dataframe.replace(np.nan, 0, inplace=True)        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.790,N/A,0.450869,N/A,0.402538,N/A,0.212617,N/A,0.083240,N/A,N/A
1,AAL,17.770,N/A,-0.367349,N/A,0.191543,N/A,0.323534,N/A,0.429631,N/A,N/A
2,AAP,163.470,N/A,0.018655,N/A,0.186743,N/A,0.021918,N/A,0.007507,N/A,N/A
3,AAPL,126.610,N/A,0.844897,N/A,0.470424,N/A,0.096605,N/A,0.024480,N/A,N/A
4,ABBV,106.460,N/A,0.304297,N/A,0.184013,N/A,0.204072,N/A,0.087292,N/A,N/A
5,ABC,101.250,N/A,0.151926,N/A,0.057858,N/A,0.048643,N/A,-0.084968,N/A,N/A
6,ABMD,259.280,N/A,0.461118,N/A,0.101724,N/A,-0.042644,N/A,-0.042199,N/A,N/A
7,ABT,112.240,N/A,0.293732,N/A,0.231197,N/A,0.026375,N/A,-0.051138,N/A,N/A
8,ACN,250.020,N/A,0.232130,N/A,0.267720,N/A,0.052794,N/A,0.013228,N/A,N/A
9,ADBE,496.630,N/A,0.584709,N/A,0.234369,N/A,0.009962,N/A,0.017985,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [150]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.790,N/A,0.450869,0.883168,0.402538,0.754455,0.212617,0.657426,0.083240,0.70495,N/A
1,AAL,17.770,N/A,-0.367349,0.0336634,0.191543,0.392079,0.323534,0.847525,0.429631,0.990099,N/A
2,AAP,163.470,N/A,0.018655,0.394059,0.186743,0.386139,0.021918,0.211881,0.007507,0.382178,N/A
3,AAPL,126.610,N/A,0.844897,0.978218,0.470424,0.841584,0.096605,0.4,0.024480,0.469307,N/A
4,ABBV,106.460,N/A,0.304297,0.788119,0.184013,0.384158,0.204072,0.641584,0.087292,0.718812,N/A
5,ABC,101.250,N/A,0.151926,0.607921,0.057858,0.168317,0.048643,0.275248,-0.084968,0.0435644,N/A
6,ABMD,259.280,N/A,0.461118,0.889109,0.101724,0.243564,-0.042644,0.0910891,-0.042199,0.132673,N/A
7,ABT,112.240,N/A,0.293732,0.776238,0.231197,0.465347,0.026375,0.223762,-0.051138,0.10099,N/A
8,ACN,250.020,N/A,0.232130,0.689109,0.267720,0.548515,0.052794,0.291089,0.013228,0.407921,N/A
9,ADBE,496.630,N/A,0.584709,0.938614,0.234369,0.471287,0.009962,0.186139,0.017985,0.437624,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [151]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    #print(momentum_percentiles)
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe
                                                      

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.790,N/A,0.450869,0.883168,0.402538,0.754455,0.212617,0.657426,0.083240,0.70495,0.75
1,AAL,17.770,N/A,-0.367349,0.0336634,0.191543,0.392079,0.323534,0.847525,0.429631,0.990099,0.565842
2,AAP,163.470,N/A,0.018655,0.394059,0.186743,0.386139,0.021918,0.211881,0.007507,0.382178,0.343564
3,AAPL,126.610,N/A,0.844897,0.978218,0.470424,0.841584,0.096605,0.4,0.024480,0.469307,0.672277
4,ABBV,106.460,N/A,0.304297,0.788119,0.184013,0.384158,0.204072,0.641584,0.087292,0.718812,0.633168
5,ABC,101.250,N/A,0.151926,0.607921,0.057858,0.168317,0.048643,0.275248,-0.084968,0.0435644,0.273762
6,ABMD,259.280,N/A,0.461118,0.889109,0.101724,0.243564,-0.042644,0.0910891,-0.042199,0.132673,0.339109
7,ABT,112.240,N/A,0.293732,0.776238,0.231197,0.465347,0.026375,0.223762,-0.051138,0.10099,0.391584
8,ACN,250.020,N/A,0.232130,0.689109,0.267720,0.548515,0.052794,0.291089,0.013228,0.407921,0.484158
9,ADBE,496.630,N/A,0.584709,0.938614,0.234369,0.471287,0.009962,0.186139,0.017985,0.437624,0.508416


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [152]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.54,N/A,0.926824,0.986139,1.506959,0.99802,0.516114,0.956436,0.267193,0.948515,0.972277
1,LRCX,491.90,N/A,0.845400,0.980198,0.791824,0.974257,0.704082,0.984158,0.168710,0.871287,0.952475
2,ALB,139.42,N/A,1.194987,0.992079,0.911402,0.984158,0.477085,0.950495,0.180861,0.879208,0.951485
3,LB,39.62,N/A,1.336092,0.994059,1.570700,1,0.416464,0.930693,0.159368,0.859406,0.94604
4,AMAT,90.60,N/A,0.524059,0.924752,0.633244,0.942574,0.626373,0.978218,0.254769,0.930693,0.944059
5,ALGN,507.79,N/A,0.853481,0.982178,1.167592,0.994059,0.622047,0.976238,0.101669,0.744554,0.924257
6,XLNX,150.90,N/A,0.558044,0.930693,0.641435,0.944554,0.473529,0.948515,0.135174,0.833663,0.914356
7,URI,243.12,N/A,0.539782,0.926733,0.712685,0.952475,0.389864,0.90099,0.164340,0.865347,0.911386
8,MU,73.28,N/A,0.452205,0.885149,0.464590,0.835644,0.541854,0.970297,0.255352,0.932673,0.905941
9,TWTR,53.76,N/A,0.716093,0.962376,0.584369,0.924752,0.323125,0.845545,0.186006,0.887129,0.90495


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [153]:
portfolio_input()

Enter the size of your portfolio1000000


In [154]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/Users/sumitchhabra/miniconda3/envs/algo/lib/python3.9/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.54,767,0.926824,0.986139,1.506959,0.99802,0.516114,0.956436,0.267193,0.948515,0.972277
1,LRCX,491.90,39,0.845400,0.980198,0.791824,0.974257,0.704082,0.984158,0.168710,0.871287,0.952475
2,ALB,139.42,140,1.194987,0.992079,0.911402,0.984158,0.477085,0.950495,0.180861,0.879208,0.951485
3,LB,39.62,494,1.336092,0.994059,1.570700,1,0.416464,0.930693,0.159368,0.859406,0.94604
4,AMAT,90.60,216,0.524059,0.924752,0.633244,0.942574,0.626373,0.978218,0.254769,0.930693,0.944059
5,ALGN,507.79,38,0.853481,0.982178,1.167592,0.994059,0.622047,0.976238,0.101669,0.744554,0.924257
6,XLNX,150.90,129,0.558044,0.930693,0.641435,0.944554,0.473529,0.948515,0.135174,0.833663,0.914356
7,URI,243.12,80,0.539782,0.926733,0.712685,0.952475,0.389864,0.90099,0.164340,0.865347,0.911386
8,MU,73.28,267,0.452205,0.885149,0.464590,0.835644,0.541854,0.970297,0.255352,0.932673,0.905941
9,TWTR,53.76,364,0.716093,0.962376,0.584369,0.924752,0.323125,0.845545,0.186006,0.887129,0.90495


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [155]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [156]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [157]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}
    
for column in column_formats.keys():
    #print(column)
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1]) 
    
    

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [158]:
writer.save()